# Lab | Final regression model in "Health Care for All" Case

## Instructions
At this point, we have created a model to predict who will make a donation and who won't. But, what about the ammount of money that each person will give? In this lab, subset those that made a donation and use that subset to create a model to predict how much money will they give.

Evaluate the result of your model and estimate how much better the result are for the bussiness in comparison with the naive scenario we discuss on Monday.

## Importing and checking if data is clean

In [64]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn import model_selection
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error


In [52]:
numerical = pd.read_csv('numerical.csv')
categorical = pd.read_csv('categorical.csv')
targets = pd.read_csv('target.csv')
data = pd.concat([numerical, categorical, targets], axis = 1)
data['TARGET_B'].value_counts()

0    90569
1     4843
Name: TARGET_B, dtype: int64

In [53]:
na = pd.DataFrame(data.isna().sum(), columns=['NA_status'])
na = na[na['NA_status']!=0]
na

,NA_status


In [54]:
donations = data[data['TARGET_B']==1].reset_index(drop=True)
donations

,TCODE,AGE,INCOME,WEALTH1,HIT,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,STATEGOV,FEDGOV,WEALTH2,POP901,POP902,POP903,POP90C1,POP90C2,POP90C3,POP90C4,POP90C5,ETH1,ETH2,ETH3,ETH4,ETH5,ETH6,ETH7,ETH8,ETH9,ETH10,ETH11,ETH12,ETH13,ETH14,ETH15,ETH16,AGE901,AGE902,AGE903,AGE904,AGE905,AGE906,AGE907,CHIL1,CHIL2,CHIL3,AGEC1,AGEC2,AGEC3,AGEC4,AGEC5,AGEC6,AGEC7,CHILC1,CHILC2,CHILC3,CHILC4,CHILC5,HHAGE1,HHAGE2,HHAGE3,HHN1,HHN2,HHN3,HHN4,HHN5,HHN6,MARR1,MARR2,MARR3,MARR4,HHP1,HHP2,DW1,DW2,DW3,DW4,DW5,DW6,DW7,DW8,DW9,HV1,HV2,HV3,HV4,HU1,HU2,HU3,HU4,HU5,HHD1,HHD2,HHD3,HHD4,HHD5,HHD6,HHD7,HHD8,HHD9,HHD10,HHD11,HHD12,ETHC1,ETHC2,ETHC3,ETHC4,ETHC5,ETHC6,HVP1,HVP2,HVP3,HVP4,HVP5,HVP6,HUR1,HUR2,RHP1,RHP2,RHP3,RHP4,HUPA1,HUPA2,HUPA3,HUPA4,HUPA5,HUPA6,HUPA7,RP1,RP2,RP3,RP4,MSA,ADI,DMA,IC1,IC2,IC3,IC4,IC5,IC6,IC7,IC8,IC9,IC10,IC11,IC12,IC13,IC14,IC15,IC16,IC17,IC18,IC19,IC20,IC21,IC22,IC23,HHAS1,HHAS2,HHAS3,HHAS4,MC1,MC2,MC3,TPE1,TPE2,TPE3,TPE4,TPE5,TPE6,TPE7,TPE8,TPE9,PEC1,PEC2,TPE10,TPE11,TPE12,TPE13,LFC1,LFC2,LFC3,LFC4,LFC5,LFC6,LFC7,LFC8,LFC9,LFC10,OCC1,OCC2,OCC3,OCC4,OCC5,OCC6,OCC7,OCC8,OCC9,OCC10,OCC11,OCC12,OCC13,EIC1,EIC2,EIC3,EIC4,EIC5,EIC6,EIC7,EIC8,EIC9,EIC10,EIC11,EIC12,EIC13,EIC14,EIC15,EIC16,OEDC1,OEDC2,OEDC3,OEDC4,OEDC5,OEDC6,OEDC7,EC1,EC2,EC3,EC4,EC5,EC6,EC7,EC8,SEC1,SEC2,SEC3,SEC4,SEC5,AFC1,AFC2,AFC3,AFC4,AFC5,AFC6,VC1,VC2,VC3,VC4,ANC1,ANC2,ANC3,ANC4,ANC5,ANC6,ANC7,ANC8,ANC9,ANC10,ANC11,ANC12,ANC13,ANC14,ANC15,POBC1,POBC2,LSC1,LSC2,LSC3,LSC4,VOC1,VOC2,VOC3,HC1,HC2,HC3,HC4,HC5,HC6,HC7,HC8,HC9,HC10,HC11,HC12,HC13,HC14,HC15,HC16,HC17,HC18,HC19,HC20,HC21,MHUC1,MHUC2,AC1,AC2,CARDPROM,NUMPROM,CARDPM12,NUMPRM12,RAMNTALL,NGIFTALL,CARDGIFT,MINRAMNT,MAXRAMNT,LASTGIFT,TIMELAG,AVGGIFT,CONTROLN,HPHONE_D,RFA_2F,CLUSTER2,STATE,CLUSTER,HOMEOWNR,GENDER,DATASRCE,RFA_2R,RFA_2A,GEOCODE2,DOMAIN_A,DOMAIN_B,ODATEW_YR,ODATEW_MM,DOB_YR,DOB_MM,MINRDATE_YR,MINRDATE_MM,MAXRDATE_YR,MAXRDATE_MM,LASTDATE_YR,LASTDATE_MM,FIRSTDATE_YR,FIRSTDATE_MM,TARGET_B,TARGET_D
0,2,62.000000,3,8,10,2,25,40,27,11,4,1,9,2707,672,929,99,0,0,45,55,97,1,0,2,1,0,0,0,0,1,0,0,1,0,0,0,35,42,43,37,47,50,27,34,43,23,9,20,27,16,8,6,15,15,13,37,21,14,16,8,15,23,30,47,29,11,2,55,10,13,22,190,267,89,49,1,10,10,8,8,8,0,902,960,8,8,88,12,97,3,11,42,72,60,34,79,21,8,1,7,12,21,5,21,58,17,0,1,0,0,4,35,74,99,0,1,64,63,62,14,4,2,8,0,5,1,7,0,95,96,96,96,5120.0,107.0,613.0,468,501,540,580,20328,6,8,19,23,23,11,7,1,2,4,6,13,27,25,14,8,1,1,14,2,66,1,61,39,20,74,17,2,2,0,0,0,2,4,1,75,20,21,1,67,71,82,62,80,60,80,65,99,0,1,21,20,8,11,17,1,1,10,0,5,3,2,1,0,0,3,23,5,2,4,12,7,3,2,1,14,12,7,5,11,4,1,4,70,10,0,140,5,3,24,21,9,32,7,3,23,3,16,7,1,2,0,12,25,3,40,20,27,7,0,0,2,12,0,0,3,1,4,1,0,0,0,4,0,1,58,95,1,0,4,95,69,23,4,13,2,17,35,91,99,1,12,3,93,0,5,1,0,1,99,1,97,99,99,9,2,6,3,26,65,5,12,61.00,15,10,2.00,7.0,5.0,12,4.066667,82943,1,3,3,other,12,H,F,3,L,D,A,S,1,87,1,36,1,88,1,94,4,96,3,87,1,1,4.0
1,0,61.611649,5,9,0,1,37,58,16,8,1,5,9,2147,591,640,99,0,0,49,51,94,2,0,3,5,1,0,1,1,0,0,0,3,0,0,2,29,36,37,27,38,40,37,46,37,17,8,34,34,15,6,2,1,18,21,31,19,11,5,0,3,7,23,71,47,15,6,77,5,1,17,286,334,99,99,0,0,0,0,0,0,0,1002,1166,7,5,97,3,96,4,3,63,92,88,59,97,3,4,1,3,5,4,1,30,60,4,1,2,0,4,15,50,94,99,2,0,91,70,72,20,4,0,0,0,3,0,0,0,78,83,99,99,2920.0,201.0,618.0,633,638,652,663,19703,2,5,1,17,48,19,7,0,1,3,4,0,16,51,19,7,0,1,5,0,65,3,74,26,25,89,9,0,0,0,0,0,1,2,1,82,34,42,22,70,82,98,67,97,66,68,61,99,99,0,34,23,4,10,12,0,2,9,0,4,1,1,0,0,4,2,25,5,2,4,8,8,8,0,3,10,8,9,4,8,1,5,5,76,4,1,160,0,1,12,25,10,39,13,5,33,6,25,7,1,1,0,19,37,2,58,11,16,10,0,4,1,15,0,0,3,2,0,2,0,0,0,1,0,3,48,97,1,1,1,99,91,24,3,6,6,57,85,95,95,5,0,0,75,0,20,0,0,5,96,4,90,99,99,12,4,1,6,10,24,0,2,68.00,11,6,2.00,10.0,7.0,9,6.181818,190313,1,3,14,TX,35,H,M,3,L,D,A,T,1,90,1,0,2,90,4,93,1,95,12,90,4,1,7.0
2,0,66.000000,5,9,5,0,33,24,39,6,5,1,9,2160,683,900,89,0,11,48,52,99,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,42,47,49,40,48,50,20,33,42,25,7,14,23,21,17,13,5,14,13,33,22,18,24,7,22,22,43,35,19,6,1,72,7,5,16,165,240,69,58,1,28,27,4,0,0,0,1282,1399,5,4,96,4,

# Modeling

### X-y split

In [55]:
y = donations['TARGET_D']
X = donations.drop(['TARGET_B', 'TARGET_D'], axis = 1)

numericalX = X.select_dtypes(np.number)
categoricalX = X.select_dtypes(object)

scaler = MinMaxScaler()
numerical_scaled = scaler.fit_transform(numericalX)
numerical_scaled = pd.DataFrame(numerical_scaled)

encoder = OneHotEncoder(drop='first').fit(categoricalX)
encoded_categorical = encoder.transform(categoricalX).toarray()
encoded_categorical = pd.DataFrame(encoded_categorical)

X = pd.concat([numerical_scaled, encoded_categorical], axis = 1)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [56]:
X

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
0,0.000051,0.617021,0.333333,0.888889,0.041667,0.020833,0.316456,0.404040,0.272727,0.229167,0.08,0.016667,1.000000,0.044224,0.044714,0.034357,1.00000,0.000000,0.000000,0.454545,0.723684,0.979798,0.010101,0.000000,0.022222,0.010204,0.000000,0.000000,0.000000,0.000000,0.021739,0.0,0.0,0.010870,0.0,0.0,0.000000,0.443038,0.531646,0.5375,0.486842,0.618421,0.657895,0.380282,0.343434,0.434343,0.232323,0.098901,0.202020,0.272727,0.470588,0.205128,0.115385,0.227273,0.151515,0.393939,0.552239,0.42,0.141414,0.163265,0.097561,0.154639,0.244681,0.352941,0.474747,0.292929,0.111111,0.020202,0.555556,0.101010,0.232143,0.239130,0.292308,0.381429,0.898990,0.494949,0.015152,0.101010,0.101010,0.080808,0.080808,0.103896,0.000000,0.150786,0.161480,0.615385,0.615385,0.888889,0.121212,0.979798,0.030612,0.111111,0.424242,0.727273,0.606061,0.343434,0.797980,0.212121,0.137931,0.1,0.125000,0.146341,0.280000,0.089286,0.295775,0.585859,0.175258,0.000000,0.014493,0.000000,0.000000,0.040404,0.353535,0.747475,1.000000,0.000000,0.011111,0.646465,0.750000,0.704545,0.229508,0.285714,0.021978,0.080808,0.000000,0.050505,0.014085,0.070707,0.000000,0.959596,0.969697,0.969697,0.969697,0.547009,0.165891,0.695800,0.312000,0.334000,0.387374,0.416069,0.116477,0.061856,0.145455,0.404255,0.389831,0.232323,0.207547,0.241379,0.047619,0.024390,0.040404,0.060606,0.131313,0.272727,0.252525,0.333333,0.258065,0.043478,0.011494,0.141414,0.036364,0.680412,0.013889,0.616162,0.419355,0.210526,0.747475,0.171717,0.038462,0.045455,0.0,0.0,0.000000,0.022222,0.102564,0.013333,0.815217,0.222222,0.388889,0.02,0.676768,0.717172,0.828283,0.626263,0.808081,0.606061,0.808081,0.656566,1.000000,0.000000,0.010101,0.269231,0.392157,0.080808,0.164179,0.250000,0.043478,0.02,0.135135,0.000000,0.104167,0.073171,0.04,0.04,0.000000,0.000000,0.053571,0.365079,0.083333,0.095238,0.08,0.141176,0.132075,0.063830,0.033333,0.016393,0.141414,0.166667,0.129630,0.135135,0.305556,0.08,0.016667,0.056338,0.707071,0.212766,0.000000,0.823529,0.050505,0.053571,0.352941,0.446809,0.290323,0.571429,0.118644,0.037037,0.333333,0.272727,0.363636,0.081395,0.012346,0.021505,0.000000,0.24,0.294118,0.136364,0.404040,0.202020,0.272727,0.070707,0.000000,0.000000,0.064516,0.171429,0.00,0.000000,0.073171,0.025641,0.058824,0.022727,0.000000,0.000000,0.000000,0.2500,0.000,0.011628,0.585859,0.959596,0.010101,0.000000,0.050633,0.959596,0.696970,0.315068,0.129032,0.250000,0.020202,0.171717,0.353535,0.919192,1.000000,0.010101,0.136364,0.083333,0.939394,0.000000,0.050505,0.011364,0.0,0.011765,1.000000,0.010101,0.979798,1.0,1.000000,0.428571,0.4,0.214286,0.06,0.489796,0.355030,0.294118,0.175439,0.021053,0.155556,0.243902,0.004444,0.002010,0.011111,0.155844,0.004033,0.432457,1.0,0.666667,0.0327

In [57]:
display(X_train.shape)
display(y_train.shape)

(3874, 354)

(3874,)

In [58]:
display(X_test.shape)
display(y_test.shape)

(969, 354)

(969,)

In [59]:
%%time
import warnings
warnings.filterwarnings('ignore')

model1 = DecisionTreeRegressor(max_depth=10,
                             criterion = 'squared_error',
                             min_samples_split=10,
                             min_samples_leaf = 10,
                             max_features = 3)

model2 = LinearRegression()

model3 = KNeighborsRegressor(n_neighbors=5)

model4= RandomForestRegressor(max_depth=None,
                             min_samples_split=10,
                             min_samples_leaf =10,
                             max_samples=0.2,
                             random_state = 42)


model_pipeline = [model1, model2, model3, model4]
model_names = ['Decision Tree Regressor', 'Linear Regression', 'KNN', 'Random Forest Regressor']
scores = {}
for model, model_name in zip(model_pipeline, model_names):
    mean_score = np.mean(cross_val_score(model, X_train, y_train, cv=5))
    scores[model_name] = mean_score
print(scores)

{'Decision Tree Regressor': -0.0497819694707329, 'Linear Regression': 0.3820665682282202, 'KNN': 0.14229416283762378, 'Random Forest Regressor': 0.5098460834226948}
CPU times: total: 29.6 s
Wall time: 24.3 s


In [60]:
%%time
val_scores = {}

for model, model_name in zip(model_pipeline,model_names):
    model.fit(X_train, y_train)
    val_scores[model_name] = model.score(X_test,y_test)    

print(val_scores)

{'Decision Tree Regressor': 0.06412263199999324, 'Linear Regression': 0.28955295013436244, 'KNN': 0.16047864185481509, 'Random Forest Regressor': 0.45119440213841455}
CPU times: total: 7.94 s
Wall time: 6.35 s


In [61]:
%%time

# Moving on with Random Forest

model4= RandomForestRegressor(max_depth=None,
                             min_samples_split=10,
                             min_samples_leaf =10,
                             max_samples=0.2,
                             random_state = 42)

model4 = model4.fit(X_train, y_train)

predicted_donations = model4.predict(X)
predicted_donations = pd.DataFrame(predicted_donations)
predicted_donations.columns=['predicted']
predicted_donations

CPU times: total: 6.06 s
Wall time: 6.07 s


,predicted
0,5.170108
1,7.858643
2,5.638494
3,14.205062
4,12.679534
...,...
4838,15.112312
4839,14.872414
4840,19.122920
4841,17.749108


In [62]:
donated_predicted = pd.concat([donations, predicted_donations],axis=1)
donated_predicted = donated_predicted[['TARGET_D', 'predicted']]
donated_predicted

,TARGET_D,predicted
0,4.0,5.170108
1,7.0,7.858643
2,5.0,5.638494
3,13.0,14.205062
4,10.0,12.679534
...,...,...
4838,20.0,15.112312
4839,15.0,14.872414
4840,3.0,19.122920
4841,10.0,17.749108


In [67]:
display(donated_predicted['TARGET_D'].mean())
display(donated_predicted['predicted'].mean())

15.624344414619037

15.61393514170983

The model can still be improved, since the score is just above 0.5, but the predicted mean is already much closer to the initial prediction, where also the donations with value zero were included.